In [1]:
import torch
device = torch.device('cuda')

Load the TRT optimized models by executing the cell below

In [2]:
import torch
import torchvision
from torch2trt import TRTModule

model_trt_collision = TRTModule()
model_trt_collision.load_state_dict(torch.load('best_model_trt.pth'))
model_trt_collision.cuda().half().eval()

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('updated_model_trt.pth')) # well trained road following model

<All keys matched successfully>

In [3]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np
from utils import preprocess

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess_col(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

In [4]:
from jetracer.nvidia_racecar import NvidiaRacecar

robot = NvidiaRacecar()

WARNNIG: Jetson.GPIO library has not been verified with this carrier board,


In [5]:
from jetcam.csi_camera import CSICamera

camera = CSICamera(width=224, height=224, capture_fps=65)
camera.running = True

In [6]:
import cv2
import ipywidgets
import threading

state_widget = ipywidgets.ToggleButtons(options=['On', 'Off'], description='Camera', value='On')
prediction_widget = ipywidgets.Image(format='jpeg', width=camera.width, height=camera.height)

live_execution_widget = ipywidgets.VBox([
    prediction_widget,
    state_widget
])

In [7]:
import traitlets
from IPython.display import display
from ipywidgets import Layout, Button, Box
import ipywidgets.widgets as widgets

network_output_slider = widgets.FloatSlider(description='Network Output', min=-1.0, max=1.0, value=0, step=0.01, orientation='horizontal', disabled=False, layout={'width': '400px'})
steering_gain_slider  = widgets.FloatSlider(description='Steering Gain', min=-1.0, max=1.0, value=-0.7, step=0.01, orientation='horizontal', layout={'width': '300px'})
steering_bias_slider  = widgets.FloatSlider(description='Steering Bias', min=-0.5, max=0.5, value=0.0, step=0.01, orientation='horizontal', layout={'width': '300px'})
steering_value_slider = widgets.FloatSlider(description='Steering', min=-1.0, max=1.0, value=0, step=0.01, orientation='horizontal', disabled=False, layout={'width': '400px'})
throttle_slider = widgets.FloatSlider(description='Throttle', min=-1.0, max=1.0, value=0, step=0.01, orientation='vertical')
prob_blocked_slider = widgets.FloatSlider(description='Probability Blocked', min=0, max=1.0, value=0, step=0.01, orientation='horizontal', disabled=False, layout={'width': '400px'})


steering_gain_link   = traitlets.link((steering_gain_slider, 'value'), (robot, 'steering_gain'))
steering_offset_link = traitlets.link((steering_bias_slider, 'value'), (robot, 'steering_offset'))
#steering_value_link  = traitlets.link((steering_value_slider, 'value'), (car, 'steering'))
throttle_slider_link = traitlets.link((throttle_slider, 'value'), (robot, 'throttle'))

display(
    widgets.HBox(
        [widgets.VBox([network_output_slider,
                       widgets.Label(value="X"),
                       steering_gain_slider,
                       widgets.Label(value="+"),
                       steering_bias_slider,
                       widgets.Label(value="||"), 
                       steering_value_slider, prob_blocked_slider], layout=Layout(
                                                    align_items='center'
                                                        )
                     ), 
         live_execution_widget,
         throttle_slider]
    )
)

In [8]:
from utils import preprocess
from jetcam.utils import bgr8_to_jpeg
go_on = 1
count_stops = 10
speed_value = 0

def update(change):
    global blocked_slider, robot,go_on,count_stops,speed_value
    a = change['new'] 
    a = preprocess_col(a)
    b = model_trt_collision(a)
    new_image = change['new'] 
    
    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    b = F.softmax(b, dim=1)
     
    
    prob_blocked = float(b.flatten()[0])
    prob_blocked_slider.value = prob_blocked
    image = preprocess(new_image).half()
    output = model_trt(image).detach().cpu().numpy().flatten()
    x = float(output[0])
    y = float(output[0])
    #blocked_slider.value = prob_blocked    
    stop_time=10#stopduration_slider.value
    
    if go_on == 1:    
        if prob_blocked > 0.8:#blocked_threshold.value: # threshold should be above 0.5
            count_stops += 1
            go_on = 2
        else:
            #start of road following detection
            go_on = 1
            count_stops = 0
    
            network_output_slider.value = x
            steering = x * steering_gain_slider.value + steering_bias_slider.value
            if(steering<-1.0):
                steering_value_slider.value = -1.0
            elif(steering>1.0):
                steering_value_slider.value = 1.0
            else:
                steering_value_slider.value = steering
            robot.steering = x
            speed_value = throttle_slider.value
    else:
        count_stops += 1
        if count_stops < stop_time:
            #x = 0.0 #set x steering to zero
            #y = 0.0 #set y steering to zero
            speed_value = 0 # set speed to zero (can set to turn as well)
        else:
            go_on = 1
            count_stops = 0
    robot.throttle = speed_value
    if(state_widget.value == 'On'):
        x = int(camera.width * (x / 2.0 + 0.5))
        y = int(camera.height * (y / 2.0 + 0.5))  
        prediction = new_image.copy()
        prediction = cv2.circle(prediction, (x, y), 8, (255, 0, 0), 3)
        prediction_widget.value = bgr8_to_jpeg(prediction)
        
update({'new': camera.value})  # we call the function once to initialize

TypeError: sub_(): argument 'other' (position 1) must be Tensor, not numpy.ndarray

In [ ]:
camera.observe(update, names='value')

In [23]:
import time

camera.unobserve(update, names='value')

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing